## Setup

In [1]:
import vibe_widget as vw
import pandas as pd
import numpy as np
import os

vw.models()

Model selection (OpenRouter)
Defaults:
  default: google/gemini-3-flash-preview
  vw.config(model="openrouter")  # -> google/gemini-3-flash-preview
  vw.config(model="openrouter", mode="premium")  # -> google/gemini-3-pro-preview
Explicit examples:
  vw.config(model="google/gemini-3-flash-preview")
  vw.config(model="google/gemini-3-pro-preview", mode="premium")
Pinned options:
  standard: ['google/gemini-3-flash-preview', 'google/gemini-2.5-flash', 'anthropic/claude-haiku-4.5', 'openai/gpt-5.1-codex-mini']
  premium:  ['google/gemini-3-pro-preview', 'anthropic/claude-opus-4.5', 'openai/gpt-5.1-codex']
More: `vw.models(show="all")` or `vw.models(verbose=False)`.

Tip: set OPENROUTER_API_KEY in your environment.



ModelsCatalog(standard=4, premium=3, latest=353)  # Use dict(vw.models(...)) to see full data

In [2]:
vw.config(
    model="google/gemini-3-flash-preview",
    api_key=os.getenv("OPENROUTE_API_KEY")
)

Config(model='google/gemini-3-flash-preview', api_key='****', temperature=0.7, streaming=True, mode='standard', theme=None, execution='auto')

## Example 1: Linked Scatter Plot & Histogram

Let's create two widgets that communicate:
1. **Scatter plot** with brush selection (outputs `selected_indices`)
2. **Histogram** that highlights selected points (inputs `selected_indices`)

When you brush-select points in the scatter plot, the histogram automatically updates!

In [3]:
data = pd.read_csv("../testdata/seattle-weather.csv")
data.head()

FileNotFoundError: [Errno 2] No such file or directory: '../testdata/seattle-weather.csv'

In [5]:
scatter = vw.create(
   description="temperature across days in Seattle, colored by weather condition and sized by wind pricipitation",
   inputs=data,
   outputs=vw.outputs(
      selected_indices=vw.export("List of selected point indices")
   ),
)

scatter

In [6]:
# Widget 2: Bar Chart showing count of records for selected weather conditions
bars = vw.create(
   "horizontal bar chart of weather conditions' count of records for selected points",
   vw.inputs(    # for multiple inputs
      data,
      selected_indices=scatter.outputs.selected_indices
   ),
)

bars

In [7]:
bars2 = bars.edit(
    "adding corresponding weather condition icon to each bar with transition ",
    vw.inputs(
      data,
      selected_indices=scatter.outputs.selected_indices
   ),
)

bars2

### How It Works

```python
# Widget A outputs a trait
scatter = vw.create(
    ...,
    vw.outputs(  # for multiple outputs
        selected_indices=scatter.outputs.selected_indices
    )
)

# Widget B inputs that trait
hist = vw.create(
    ...,
    vw.inputs(  # for multiple inputs
        df,
        selected_indices=scatter.outputs.selected_indices
    )
)

# Widget B-1 edit based on the Widget B
hist2 = hist.edit(
    ...,
    vw.inputs(
        df,
        selected_indices=scatter.outputs.selected_indices
    )
)
```

Vibe Widget automatically:
1. Creates the trait on the exporting widget
2. Links it bidirectionally using traitlets
3. Updates all importing widgets when the trait changes
4. Generates code that listens for trait changes

## Example 2: 2D Terrain Painter → 3D Landscape

This is one of the coolest examples! We'll create:
1. **2D Canvas**: Paint terrain height with your mouse (outputs `heightmap`)
2. **3D Viewer**: Renders the terrain in 3D using Three.js (inputs `heightmap`)

Paint on the canvas and watch it render in 3D in real-time!

In [23]:
# Widget 1: 2D terrain painter
painter = vw.create(
    """2D canvas for painting terrain height with mouse brush.
    - Click and drag to paint elevation
    - Store heightmap as 64x64 grid of floats 0-1
    - Show a gradient from blue (low) to white (high)
    - Use very high intensity brushes for dramatic terrain
    """,
    outputs={
        "heightmap": "64x64 grid of float values (0-1) representing terrain elevation"
    }
)

print("\n2D Terrain painter created!")
print("Click and drag to paint terrain")


2D Terrain painter created!
Click and drag to paint terrain


In [11]:
# Widget 2: 3D landscape viewer
landscape = vw.create(
    """3D landscape viewer using Three.js (import via ESM).
    - Create terrain mesh from heightmap (64x64 grid)
    - Use PlaneGeometry and displace vertices based on heightmap values
    - Add orbit controls for rotation and zoom
    - show the terrain as mountains above a water plane at level 2 (height = intensity*5)
    - Add a water plane at level 2
    - Update mesh when heightmap changes
    """,
    inputs={
        "heightmap": painter
    }
)

print("\n3D Landscape viewer created and linked!")
print("Paint on canvas → See in 3D instantly")


3D Landscape viewer created and linked!
Paint on canvas → See in 3D instantly


### Bonus: Simulate Rain Erosion

Let's add a button that simulates hydraulic erosion on the terrain!

In [12]:
import ipywidgets as widgets
from IPython.display import display

GRID_SIZE = 64

def simulate_erosion(b):
    """Simulate hydraulic erosion on the terrain"""
    btn.description = "🌧️ Raining..."
    btn.disabled = True
    
    # Get current heightmap
    h_list = painter.heightmap or [0.0] * (GRID_SIZE * GRID_SIZE)
    grid = np.array(h_list).reshape((GRID_SIZE, GRID_SIZE))
    
    # Simulation parameters
    drops = 2000
    erosion_rate = 0.01
    deposition_rate = 0.005
    
    # Simple hydraulic erosion
    for _ in range(drops):
        # Random raindrop start position
        x, y = np.random.randint(0, GRID_SIZE, 2)
        
        path_len = 0
        while path_len < 30:  # Max path length
            path_len += 1
            
            # Find lowest neighbor
            best_nx, best_ny = x, y
            min_h = grid[y, x]
            
            for dy in [-1, 0, 1]:
                for dx in [-1, 0, 1]:
                    if dx == 0 and dy == 0:
                        continue
                    nx, ny = x + dx, y + dy
                    if 0 <= nx < GRID_SIZE and 0 <= ny < GRID_SIZE:
                        if grid[ny, nx] < min_h:
                            min_h = grid[ny, nx]
                            best_nx, best_ny = nx, ny
            
            if best_nx == x and best_ny == y:
                # Local minimum (pool) - deposit sediment
                grid[y, x] += deposition_rate
                break
            else:
                # Flow downhill - erode current position
                grid[y, x] -= erosion_rate
                x, y = best_nx, best_ny
    
    # Clip and update widget
    grid = np.clip(grid, 0.0, 1.0)
    painter.heightmap = grid.flatten().tolist()
    
    btn.description = "🌧️ Simulate Rain Erosion"
    btn.disabled = False

btn = widgets.Button(
    description="🌧️ Simulate Rain Erosion",
    layout=widgets.Layout(width='100%', height='50px'),
    style={'button_color': '#4488ff'}
)
btn.on_click(simulate_erosion)

display(widgets.VBox([
    widgets.Label("Simulate 2000 raindrops eroding the terrain:"),
    btn
]))

## Example 3: Solar System Explorer

Create an interactive 3D solar system where clicking a planet highlights its data in a chart!

1. **3D Solar System**: Click planets to select them (outputs `selected_planet`)
2. **Bar Chart**: Shows planet data with selection highlighting (inputs `selected_planet`)

In [13]:
# Load planets data
df_planets = pd.read_csv('../testdata/planets.csv')
df_planets.head()

,planet,mass,diameter,density,gravity,escape_velocity,rotation_period,length_of_day,distance_from_sun,perihelion,...,orbital_period,orbital_velocity,orbital_inclination,orbital_eccentricity,obliquity_to_orbit,mean_temperature,surface_pressure,number_of_moons,has_ring_system,has_global_magnetic_field
0,Mercury,0.330,4879,5427,3.7,4.3,1407.6,4222.6,57.9,46.0,...,88.0,47.4,7.0,0.205,0.034,167,0,0,No,Yes
1,Venus,4.870,12104,5243,8.9,10.4,-5832.5,2802.0,108.2,107.5,...,224.7,35.0,3.4,0.007,177.400,464,92,0,No,No
2,Earth,5.970,12756,5514,9.8,11.2,23.9,24.0,149.6,147.1,...,365.2,29.8,0.0,0.017,23.400,15,1,1,No,Yes
3,Mars,0.642,6792,3933,3.7,5.0,24.6,24.7,227.9,206.6,...,687.0,24.1,1.9,0.094,25.200,-65,0.01,2,No,No
4,Jupiter,1898.000,142984,1326,23.1,59.5,9.9,9.9,778.6,740.5,...,4331.0,13.1,1.3,0.049,3.100,-110,Unknown*,79,Yes,Yes


In [ ]:
# Widget 1: 3D Solar System
solar_system = vw.create(
    """3D solar system using Three.js showing planets orbiting the sun.
    - Create spheres for each planet with relative sizes
    - Position planets at their relative distances from sun
    - Make planets clickable to select them
    - Highlight selected planet with a bright glow/outline
    - Add orbit controls for rotation
    - Default selection: Earth
    - Export the selected planet name
    """,
    inputs=df_planets,
    outputs={
        "selected_planet": "name of the currently selected planet"
    }
)

print("\n3D Solar system created!")
print("Click planets to select them")


🪐 3D Solar system created!
🖱️ Click planets to select them


In [15]:
# Monitor selection changes
import ipywidgets as widgets
from IPython.display import display

label = widgets.Label(value=f"Selected: {solar_system.outputs.selected_planet() or 'None'}")

def on_planet_change(change):
    label.value = f"Selected: {change.new or 'None'}"

solar_system.observe(on_planet_change, names='selected_planet')
display(label)

Label(value='Selected: None')

In [16]:
# Widget 2: Planet data chart
planet_chart = vw.create(
    """Bar chart showing planet properties.
    - Add dropdown to switch between metrics: distance, mass, radius, orbital_period
    - Highlight the selected planet bar with an orange border and background
    - Sort bars by the selected metric
    - Show values on bars
    - Use a clean, modern design
    """,
    inputs=vw.inputs(
        df_planets,
        selected_planet=solar_system
    )
)


print("Click planets in 3D → See highlighted in chart")

Click planets in 3D → See highlighted in chart


## Understanding Exports & Imports

### Exports

When you **export** a trait, you're saying: "This widget will expose this piece of state."

```python
widget = vw.create(
    "widget description",
    data,
    outputs={
        "trait_name": "description of what this trait contains"
    }
)
```

The AI uses your description to:
- ✅ Generate code that updates this trait
- ✅ Choose the right data type (array, string, object, etc.)
- ✅ Set up event listeners to keep it synchronized

### Imports

When you **import** a trait, you're saying: "This widget needs to react to changes in another widget."

```python
widget2 = vw.create(
    "widget description",
    data,
    inputs={
        "trait_name": source_widget
    }
)
```

The AI generates code that:
- ✅ Listens for changes to the imported trait
- ✅ Updates the visualization when the trait changes
- ✅ Handles the trait data appropriately

### Bidirectional Linking

Vibe Widget uses **traitlets** under the hood to create bidirectional links:

```
Widget A (outputs)  ←→  Widget B (inputs)
     ↓ trait change         ↑ automatically updated
```

This means changes flow automatically in both directions!

## Tic Tac Toe: AI Game with Decision Tree Visualization

**This demo showcases:**
- Train a Decision Tree model on 958 complete tic-tac-toe games
- Play against the AI in an interactive game board
- Visualize the AI's decision-making process in real-time
- Cross-widget communication between game state, AI engine, and visualization

**Dataset**: Complete set of possible board configurations where X plays first, with win/loss outcomes for X

In [18]:
# Load the proper training datasets
x_moves_df = pd.read_csv('../testdata/X_moves.csv')
o_moves_df = pd.read_csv('../testdata/O_moves.csv')

print(f"Loaded X_moves: {len(x_moves_df)} moves")
print(f"Loaded O_moves: {len(o_moves_df)} moves")
print(f"\nX_moves sample:")
x_moves_df.head()

Loaded X_moves: 9279 moves
Loaded O_moves: 9279 moves

X_moves sample:


,GameNr,MoveNr,00-1,00-2,01-1,01-2,02-1,02-2,10-1,10-2,...,12-2,20-1,20-2,21-1,21-2,22-1,22-2,move_I,move_J,winner
0,11,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,1,2
1,11,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,2,2
2,11,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2,1,2
3,11,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0,0,2
4,11,5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0,1,2


In [19]:
# Train Tic-Tac-Toe AI using Tik-Taker approach with improved parameters
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd


# Feature columns (board state encoding)
feature_cols = ['00-1', '00-2', '01-1', '01-2', '02-1', '02-2', 
                '10-1', '10-2', '11-1', '11-2', '12-1', '12-2', 
                '20-1', '20-2', '21-1', '21-2', '22-1', '22-2']

print("\nTRAINING X PLAYER (Human plays X)")
print("-" * 70)
# Prepare X training data (all winning games for X)
X_features = x_moves_df[feature_cols]
X_move_I = x_moves_df['move_I']
X_move_J = x_moves_df['move_J']

# Split for evaluation
X_train_feat, X_test_feat, X_train_I, X_test_I, X_train_J, X_test_J = train_test_split(
    X_features, X_move_I, X_move_J, test_size=0.15, random_state=42
)

print(f"Training samples: {len(X_train_feat)}, Test samples: {len(X_test_feat)}")

# Train X player models with improved parameters
print("Training X row predictor (GradientBoosting with depth 5)...")
lr_I_X = GradientBoostingClassifier(
    n_estimators=100, 
    max_depth=5, 
    learning_rate=0.1,
    random_state=42
)
lr_I_X.fit(X_train_feat, X_train_I)

print("Training X column predictor (GradientBoosting with depth 5)...")
lr_J_X = GradientBoostingClassifier(
    n_estimators=100, 
    max_depth=5, 
    learning_rate=0.1,
    random_state=42
)
lr_J_X.fit(X_train_feat, X_train_J)

# Evaluate X models
X_I_acc = lr_I_X.score(X_test_feat, X_test_I)
X_J_acc = lr_J_X.score(X_test_feat, X_test_J)
print(f"X Player - Row accuracy: {X_I_acc:.2%}, Column accuracy: {X_J_acc:.2%}")

print("\nTRAINING O PLAYER (AI plays O)")
print("-" * 70)
# Prepare O training data (only winning games for O)
o_winning = o_moves_df[(o_moves_df['winner'] == 1) & (o_moves_df['move_I'] != -1)]
print(f"Using {len(o_winning)} winning O moves out of {len(o_moves_df)} total")

O_features = o_winning[feature_cols]
O_move_I = o_winning['move_I']
O_move_J = o_winning['move_J']

# Split for evaluation
O_train_feat, O_test_feat, O_train_I, O_test_I, O_train_J, O_test_J = train_test_split(
    O_features, O_move_I, O_move_J, test_size=0.15, random_state=42
)

print(f"Training samples: {len(O_train_feat)}, Test samples: {len(O_test_feat)}")

# Train O player models with improved parameters
print("Training O row predictor (GradientBoosting with depth 5)...")
lr_I_O = GradientBoostingClassifier(
    n_estimators=100, 
    max_depth=5, 
    learning_rate=0.1,
    random_state=42
)
lr_I_O.fit(O_train_feat, O_train_I)

print("Training O column predictor (GradientBoosting with depth 5)...")
lr_J_O = GradientBoostingClassifier(
    n_estimators=100, 
    max_depth=5, 
    learning_rate=0.1,
    random_state=42
)
lr_J_O.fit(O_train_feat, O_train_J)

# Evaluate O models
O_I_acc = lr_I_O.score(O_test_feat, O_test_I)
O_J_acc = lr_J_O.score(O_test_feat, O_test_J)
print(f"O Player - Row accuracy: {O_I_acc:.2%}, Column accuracy: {O_J_acc:.2%}")


# Helper functions for board conversion
def board_to_features(board_list):
    """
    Convert board state ['x','o','b',...] to feature vector for model.
    board_list: 9-element list in order [00, 01, 02, 10, 11, 12, 20, 21, 22]
    Returns: 18-element list with one-hot encoding
    """
    features = []
    for cell in board_list:
        if cell == 'o':
            features.extend([1.0, 0.0])
        elif cell == 'x':
            features.extend([0.0, 1.0])
        else:  # 'b' for blank
            features.extend([0.0, 0.0])
    return features

def get_empty_positions(board_list):
    """Get list of empty (row, col) positions"""
    empty = []
    for idx, cell in enumerate(board_list):
        if cell == 'b':
            row = idx // 3
            col = idx % 3
            empty.append((row, col))
    return empty

def check_winning_move(board_state, player):
    """Check if there's a winning move for the player"""
    empty_positions = get_empty_positions(board_state)
    for row, col in empty_positions:
        idx = row * 3 + col
        test_board = board_state.copy()
        test_board[idx] = player
        if check_winner(test_board) == player:
            return (row, col)
    return None

def check_winner(board):
    """Check if there's a winner on the board"""
    # Check rows
    for i in range(3):
        if board[i*3] == board[i*3+1] == board[i*3+2] != 'b':
            return board[i*3]
    # Check columns
    for i in range(3):
        if board[i] == board[i+3] == board[i+6] != 'b':
            return board[i]
    # Check diagonals
    if board[0] == board[4] == board[8] != 'b':
        return board[0]
    if board[2] == board[4] == board[6] != 'b':
        return board[2]
    return None

def predict_best_move(board_state, player='o'):
    """
    Predict best move for given player using trained models.
    board_state: 9-element list ['x','o','b',...] in order [00,01,02,10,11,12,20,21,22]
    player: 'x' or 'o'
    Returns: (row, col) tuple or None if no valid moves
    """
    empty_positions = get_empty_positions(board_state)
    if not empty_positions:
        return None
    
    # First priority: Check if we can win
    winning_move = check_winning_move(board_state, player)
    if winning_move:
        return winning_move
    
    # Second priority: Block opponent's winning move
    opponent = 'x' if player == 'o' else 'o'
    blocking_move = check_winning_move(board_state, opponent)
    if blocking_move:
        return blocking_move
    
    # Convert board to features
    features = board_to_features(board_state)
    X_input = pd.DataFrame([features], columns=feature_cols)
    
    # Get model predictions
    if player == 'x':
        I_probs = lr_I_X.predict_proba(X_input)
        J_probs = lr_J_X.predict_proba(X_input)
    else:  # 'o'
        I_probs = lr_I_O.predict_proba(X_input)
        J_probs = lr_J_O.predict_proba(X_input)
    
    # Compute joint probability matrix (outer product)
    prob_matrix = np.dot(I_probs.T, J_probs)  # 3x3 matrix
    
    # Find best valid move
    best_score = -1
    best_move = None
    
    for row, col in empty_positions:
        score = prob_matrix[row, col]
        if score > best_score:
            best_score = score
            best_move = (row, col)
    
    return best_move


TRAINING X PLAYER (Human plays X)
----------------------------------------------------------------------
Training samples: 7887, Test samples: 1392
Training X row predictor (GradientBoosting with depth 5)...
Training X column predictor (GradientBoosting with depth 5)...
X Player - Row accuracy: 52.01%, Column accuracy: 50.65%

TRAINING O PLAYER (AI plays O)
----------------------------------------------------------------------
Using 9279 winning O moves out of 9279 total
Training samples: 7887, Test samples: 1392
Training O row predictor (GradientBoosting with depth 5)...
Training O column predictor (GradientBoosting with depth 5)...
O Player - Row accuracy: 47.84%, Column accuracy: 48.56%


In [21]:
# Create interactive Tic-Tac-Toe game board
game_board = vw.create(
    """Interactive Tic-Tac-Toe game board with the following features:
    - 3x3 grid with clickable cells (use CSS grid layout)
    - Human player is 'X' (blue), AI is 'O' (red)
    - Each cell shows 'X' or 'O' when occupied, empty when blank
    - Display current game status (X's turn, O's turn, X wins, O wins, Draw)
    - Reset button to start new game
    
    CRITICAL BOARD STATE FORMAT:
    - Export board_state as a 9-element array of strings
    - Each element is either 'x', 'o', or 'b' (blank)
    - Order MUST be: [row0col0, row0col1, row0col2, row1col0, row1col1, row1col2, row2col0, row2col1, row2col2]
    - This maps to positions: [00, 01, 02, 10, 11, 12, 20, 21, 22] in row-major order
    - Example: ['x','b','o','b','x','b','b','o','b'] means:
        X _ O
        _ X _
        _ O _
    
    - Export game_over as boolean (true when game ends)
    - Export current_turn as string ('x' or 'o')
    
    - Import ai_move as object {row: number, col: number} to trigger AI move
    - When ai_move changes, place 'o' at board[row*3 + col]
    
    - Automatically alternate turns between X and O after each move
    - Detect win conditions: 3 in a row horizontally, vertically, or diagonally
    - Detect draw: board full with no winner
    - Update board_state, game_over, and current_turn after every move
    """,
    outputs={
        "board_state": "array of 9 strings: ['x'|'o'|'b', ...] in row-major order",
        "game_over": "boolean indicating if game is finished",
        "current_turn": "string 'x' or 'o' indicating whose turn"
    },
    inputs={
        "ai_move": "object {row: number, col: number} for AI move placement"
    },
)

game_board

In [22]:
# Create AI controller to compute and send moves
import time

def make_ai_move(change):
    """Called when board_state or current_turn changes"""
    # Wait a bit for better UX
    time.sleep(0.3)
    
    try:
        board_state = game_board.outputs.board_state.value
        current_turn = game_board.outputs.current_turn.value
        game_over = game_board.outputs.game_over.value
        
        # Only make move if it's O's turn and game is not over
        if current_turn != 'o' or game_over or not board_state:
            return
        
        # Convert board_state to list if needed
        if isinstance(board_state, str):
            import ast
            board_state = ast.literal_eval(board_state)
        
        # Ensure it's a list
        board_list = list(board_state)
        
        # Validate board format (should be 9 elements)
        if len(board_list) != 9:
            print(f"Invalid board state length: {len(board_list)}, expected 9")
            return
        
        # The board widget outputs in row-major order: [00,01,02,10,11,12,20,21,22]
        # Our predict_best_move expects the same format
        move = predict_best_move(board_list, player='o')
        
        if move:
            print(f"AI (O) plays at position ({move[0]}, {move[1]})")
            # Send move back to widget
            game_board.ai_move = {"row": int(move[0]), "col": int(move[1])}
        else:
            print("No valid move found")
            
    except Exception as e:
        print(f"Error in AI move: {e}")
        import traceback
        traceback.print_exc()

# Observe changes to trigger AI moves
game_board.observe(make_ai_move, names=['current_turn'])